In [14]:
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
data=pd.read_csv('Churn_Modelling.csv')
data

In [ ]:
# Oreprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data

In [ ]:
# One hot encode 'Geography'
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

In [ ]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

In [ ]:
## Encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [17]:
# Split the data into features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [43]:
## Split the data in training and tetsing sets
print(X.shape)
print(y.shape)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape)
print(y_train.shape)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

(10000, 12)
(10000,)
(8000, 12)
(8000,)


In [40]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [ ]:
y_test

In [ ]:
## Save the encoders and sscaler
# Pickling” is the process whereby a Python object hierarchy is converted into a byte stream
# here we are saving to a file the obects: label_encoder_gender, onehot_encoder_geo
#with open('label_encoder_gender.pkl','wb') as file:
#    pickle.dump(label_encoder_gender,file)

#with open('onehot_encoder_geo.pkl','wb') as file:
#    pickle.dump(onehot_encoder_geo,file)

#with open('scaler.pkl','wb') as file:
#    pickle.dump(scaler,file)


## ANN Regression Problem statement

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
print(X_train.shape[0]) # rows
print(X_train.shape[1]) # columns

In [ ]:
# Build the model
# the neural tetwork with thte 2 hid
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer, 64 is decided initially
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
])

# compile the model
# https://keras.io/api/losses/
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

In [25]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Setup TensorBoard
log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [26]:
# Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)

In [ ]:
# Train the model
history=model.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]

)

In [29]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir regressionlogs/fit
#20240912-205012


In [ ]:
# Evaluate model on the test data
test_loss, test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

In [ ]:
model.save('regression_model.h5')